In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time


d:\Anaconda\envs\general_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# train_dataset=torchvision.datasets.MNIST?

In [4]:
train_transform=transforms.Compose([
    transforms.RandomRotation(degrees=30),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

test_transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset=torchvision.datasets.MNIST(
    root=r"..\dataset",
    train=True,
    transform=train_transform,
    download=True
)

test_dataset=torchvision.datasets.MNIST(
    root=r"..\dataset",
    train=False,
    transform=test_transform,
    download=True
)

train_dataloader=torch.utils.data.DataLoader(
    dataset=train_dataset,
    shuffle=True,
    batch_size=64
    )

test_dataloader=torch.utils.data.DataLoader(
    dataset=test_dataset,
    shuffle=False,
    batch_size=64
    )

for batch_idx, (images, labels) in enumerate(train_dataloader):
    images=images.to(device)
    if batch_idx==len(train_dataloader)-1:
        print(images.shape, labels.shape)



torch.Size([32, 1, 28, 28]) torch.Size([32])


In [5]:

# images, labels = next(iter(train_dataloader)) 
# print(images.shape,labels.shape)           
# fig = plt.figure(figsize=(8, 8))
# for i in range(25):
#     plt.subplot(5,5,i+1)                      
#     plt.tight_layout()                        
#     plt.imshow(images[i][0], cmap='gray', interpolation='none') 
#     plt.title("gt {}".format(labels[i]))
#     plt.xticks([])


In [6]:
from model import VGG
from train import train_model
SimpleVGG = VGG.SimpleVGG(num_classes=10, batch_norm=True, in_channels=1).to(device)



In [7]:
history_mnist_vgg, logdir_mnist_vgg = train_model(
    model=SimpleVGG,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    lr=1e-3,
    batch_size=64,
    num_epochs=20,
    weight_decay=1e-4,
    patience=4,
    min_delta=1e-5,
    log_root="./logs",
    exp_name="SimpleVGG_MNIST_lr1e-3_bs64",
    device=device,
)
print("VGG logs in:", logdir_mnist_vgg)

[VGG] Epoch [1/20] LR=0.001, BS=64 | Train Loss=0.2666, Test Loss=0.1021, Test Acc=0.9722, Time=55.71s
New best model at epoch 1 with acc 0.9722


[VGG] Epoch [2/20] LR=0.001, BS=64 | Train Loss=0.0924, Test Loss=0.0920, Test Acc=0.9735, Time=50.89s
New best model at epoch 2 with acc 0.9735


[VGG] Epoch [3/20] LR=0.001, BS=64 | Train Loss=0.0783, Test Loss=0.0625, Test Acc=0.9819, Time=49.07s
New best model at epoch 3 with acc 0.9819


[VGG] Epoch [4/20] LR=0.001, BS=64 | Train Loss=0.0719, Test Loss=0.0683, Test Acc=0.9836, Time=48.05s
New best model at epoch 4 with acc 0.9836


[VGG] Epoch [5/20] LR=0.001, BS=64 | Train Loss=0.0641, Test Loss=0.0507, Test Acc=0.9865, Time=47.92s
New best model at epoch 5 with acc 0.9865


...

[VGG] Epoch [9/20] LR=0.001, BS=64 | Train Loss=0.0501, Test Loss=0.0331, Test Acc=0.9917, Time=48.10s
New best model at epoch 9 with acc 0.9917


[VGG] Epoch [10/20] LR=0.001, BS=64 | Train Loss=0.0474, Test Loss=0.0430, Test Acc=0.9867, Time=48.28s


[VGG] Epoch [11/20] LR=0.001, BS=64 | Train Loss=0.0475, Test Loss=0.0558, Test Acc=0.9839, Time=48.17s


[VGG] Epoch [12/20] LR=0.001, BS=64 | Train Loss=0.0445, Test Loss=0.0272, Test Acc=0.9914, Time=48.22s


[VGG] Epoch [13/20] LR=0.001, BS=64 | Train Loss=0.0398, Test Loss=0.0370, Test Acc=0.9900, Time=48.16s
[VGG] Early stopping at epoch 13 (best epoch 9, best acc 0.9917)
[VGG] Total training time: 640.42s, Best Acc=0.9917 at epoch 9
VGG logs in: ./logs\SimpleVGG_MNIST_lr1e-3_bs64


In [13]:
from evaluation import evaluate_model

metrics_mnist_vgg = evaluate_model(
    model=SimpleVGG,
    checkpoint_path=history_mnist_vgg["model_best_path"],
    test_dataset=test_dataset,
    batch_size=64,
    num_classes=10,
    class_names=[str(i) for i in range(10)],
    log_dir=history_mnist_vgg["log_dir"],
    device=device,
    max_visualization_samples=4,
    image_mean=[0.1307],
    image_std=[0.3081],
)
print(metrics_mnist_vgg["metrics_txt_path"])

./logs\SimpleVGG_MNIST_lr1e-3_bs64\test_results\test_metrics.txt


In [18]:
from model import BottleNeck,ResNet
Resnet50 = ResNet(
    block=BottleNeck,
    block_num=[3, 4, 6, 3],
    num_classes=10,
    include_top=True,
    input_channel=1).to(device)

In [19]:
history_mnist_resnet, logdir_mnist_resnet = train_model(
    model=Resnet50,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    lr=1e-3,
    batch_size=64,
    num_epochs=20,
    weight_decay=1e-4,
    patience=4,
    min_delta=1e-5,
    log_root="./logs",
    exp_name="ResNet50_MNIST_lr1e-3_bs64",
    device=device,
)
print("ResNet logs in:", logdir_mnist_resnet)

[ResNet] Epoch [1/20] LR=0.001, BS=64 | Train Loss=0.3442, Test Loss=0.0881, Test Acc=0.9746, Time=134.11s
New best model at epoch 1 with acc 0.9746


[ResNet] Epoch [2/20] LR=0.001, BS=64 | Train Loss=0.1396, Test Loss=0.0836, Test Acc=0.9763, Time=134.60s
New best model at epoch 2 with acc 0.9763


[ResNet] Epoch [3/20] LR=0.001, BS=64 | Train Loss=0.1003, Test Loss=0.0451, Test Acc=0.9869, Time=134.82s
New best model at epoch 3 with acc 0.9869


[ResNet] Epoch [4/20] LR=0.001, BS=64 | Train Loss=0.0871, Test Loss=0.0430, Test Acc=0.9864, Time=134.86s


[ResNet] Epoch [5/20] LR=0.001, BS=64 | Train Loss=0.0850, Test Loss=0.0506, Test Acc=0.9838, Time=134.37s


...

[ResNet] Epoch [16/20] LR=0.001, BS=64 | Train Loss=0.0525, Test Loss=0.0435, Test Acc=0.9872, Time=134.40s


[ResNet] Epoch [17/20] LR=0.001, BS=64 | Train Loss=0.0510, Test Loss=0.0277, Test Acc=0.9914, Time=135.47s
New best model at epoch 17 with acc 0.9914


[ResNet] Epoch [18/20] LR=0.001, BS=64 | Train Loss=0.0487, Test Loss=0.0343, Test Acc=0.9890, Time=134.80s


[ResNet] Epoch [19/20] LR=0.001, BS=64 | Train Loss=0.0515, Test Loss=0.0348, Test Acc=0.9896, Time=135.46s


[ResNet] Epoch [20/20] LR=0.001, BS=64 | Train Loss=0.0460, Test Loss=0.0300, Test Acc=0.9910, Time=134.76s
[ResNet] Total training time: 2705.91s, Best Acc=0.9914 at epoch 17
ResNet logs in: ./logs\ResNet50_MNIST_lr1e-3_bs64


In [20]:
from evaluation import evaluate_model

metrics_mnist_resnet = evaluate_model(
    model=Resnet50,
    checkpoint_path=history_mnist_resnet["model_best_path"],
    test_dataset=test_dataset,
    batch_size=64,
    num_classes=10,
    class_names=[str(i) for i in range(10)],
    log_dir=history_mnist_resnet["log_dir"],
    device=device,
    max_visualization_samples=8,
    image_mean=[0.1307],
    image_std=[0.3081],
)
print(metrics_mnist_resnet["metrics_txt_path"])

./logs\ResNet50_MNIST_lr1e-3_bs64\test_results\test_metrics.txt
